In [4]:
import cv2
import numpy as np
import os

def preprocess_image(image_path):
    # Load the image and convert to grayscale
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply binary thresholding (adaptive if necessary)
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    return binary, image  # Return both binary and original image for contour display

def sort_contours(contours):
    # Sort contours based on their position: top-to-bottom, then left-to-right
    bounding_boxes = [cv2.boundingRect(c) for c in contours]
    # Sort by y (top-to-bottom), and within each row, sort by x (left-to-right)
    sorted_contours = sorted(zip(contours, bounding_boxes), key=lambda b: (b[1][1], b[1][0]))
    return [c for c, _ in sorted_contours]

def extract_letters(binary_image, original_image, output_folder):
    # Find contours of the letters
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort the contours in reading order
    sorted_contours = sort_contours(contours)

    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    letter_count = 0

    for contour in sorted_contours:
        # Get the bounding box for each letter
        x, y, w, h = cv2.boundingRect(contour)

        # Ignore small contours (noise)
        if w < 20 or h < 20:
            continue

        # Draw bounding box on the original image for visualization
        cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Crop the letter from the binary image
        letter = binary_image[y:y + h, x:x + w]

        # Resize the letter to 80x80 pixels
        letter_resized = cv2.resize(letter, (80, 80))

        # Save the cropped letter
        letter_filename = f"{output_folder}/letter_{letter_count:03}.png"
        cv2.imwrite(letter_filename, letter_resized)

        # Display the letter
        cv2.imshow(f"Letter {letter_count}", letter_resized)
        cv2.waitKey(500)  # Display each letter for 500ms

        letter_count += 1

    # Display the original image with contours
    cv2.imshow("Detected Contours", original_image)
    cv2.waitKey(0)  # Wait until any key is pressed
    cv2.destroyAllWindows()

    print(f"Extracted {letter_count} letters and saved them in '{output_folder}'.")

if __name__ == "__main__":
    # Specify the input image and output folder
    input_image_path = "../Sentences/adapanthipawathwannena/adapanthipawathwannena.png"
    output_folder = "../Sentences/adapanthipawathwannena"

    # Preprocess the image to get a binary version and original image
    binary_image, original_image = preprocess_image(input_image_path)

    # Extract and save letters, and display results
    extract_letters(binary_image, original_image, output_folder)


Extracted 165 letters and saved them in '../Sentences/adapanthipawathwannena'.
